# Qiskit Comparison
This notebook highlights how my simulator is a nice little subset of qiskit's simulator.

## Qiskit Circuit
To validate my implementation, first we create a simple circuit in qiskit and run simluation on it.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.h(0)
qc.h(1)
qc.cz(0, 4)
qc.unitary([[0.5 + 0.5j, 0.5 - 0.5j], [0.5 - 0.5j, 0.5 + 0.5j]], 1)
out = qc.measure([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])
qc.draw()


┌───┐              ┌─┐      
q_0: ┤ H ├────────────■─┤M├──────
     ├───┤┌─────────┐ │ └╥┘┌─┐   
q_1: ┤ H ├┤ unitary ├─┼──╫─┤M├───
     └┬─┬┘└─────────┘ │  ║ └╥┘   
q_2: ─┤M├─────────────┼──╫──╫────
      └╥┘     ┌─┐     │  ║  ║    
q_3: ──╫──────┤M├─────┼──╫──╫────
       ║      └╥┘     │  ║  ║ ┌─┐
q_4: ──╫───────╫──────■──╫──╫─┤M├
       ║       ║         ║  ║ └╥┘
c: 5/══╩═══════╩═════════╩══╩══╩═
       2       3         0  1  4

In [2]:
from qiskit import Aer, execute
import time

def simulate(circuit, num_shots):
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(circuit, backend_sim, shots=num_shots)
    result_sim = job_sim.result()
    return result_sim.get_counts(circuit)

start = time.time()
qiskit_counts = simulate(qc, num_shots=10000)
qiskit_diff = time.time() - start

ImportError: cannot import name 'executee' from 'qiskit' (/usr/lib/python3.9/site-packages/qiskit/__init__.py)

## JQiskit Circuit
Here is the equivalent simulator in `jqiskit`. Notice how the API is pretty much the same, the main difference is that the `qc.measure()` function does both the simulation and state generation.

In [ ]:
from jqiskit.api import QuantumCircuit as JQuantumCircuit

qc = JQuantumCircuit(5)
qc.h(0)
qc.h(1)
qc.cz(0, 4)
qc.sqrtnot(1)

start = time.time()
jqiskit_counts, state = qc.measure(num_shots=10000)
jqiskit_diff = time.time() - start

## Timing Comparison

This clearly isn't a fair fight, as qiskit is probably doing a bunch of stuff under the hood for more complex features that I'm not doing for this simple simulator. But for what it's worth the simple version is significantly faster for the circuit described above. This may indicate a need for a more light-weight framework in the future for faster hyper parameter searches and stuff like that. For example, you can imagine a world where variational optimizations are first done on a faster, more ideal solver and then fine-tuned on a slower more accuate one.

In [ ]:
print(f'jqiskit_speed: {jqiskit_diff * 1000:.2f} ms')
print(f' qiskit_speed: {qiskit_diff * 1000:.2f} ms')

## Validation

I then compared the two by looking at the simulation output states. Note how qiskit is the opposite endian-ness as my implementation, so I had to flip the state to do a proper comparison.

In [ ]:
assert len(jqiskit_counts) == len(qiskit_counts), "Number of states don't match!"

for state_str in jqiskit_counts:
    print(f'state: {state_str}, qiskit: {qiskit_counts[state_str[::-1]]}; jqiskit: {jqiskit_counts[state_str]}')